In [1]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time

In [16]:
# from recitation-6
imageFolder_dataset = torchvision.datasets.ImageFolder(root='./train_data/medium/', 
                                                       transform=torchvision.transforms.ToTensor())
imageFolder_dataloader = DataLoader(imageFolder_dataset, batch_size=1, shuffle=False, num_workers=1)

# Inverted Residual Block

I have taken MobileNetV2 implementation from this gitHub repository
https://github.com/pytorch/vision/blob/master/torchvision/models/mobilenet.py

In [2]:
class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            nn.ReLU6(inplace=True)
        )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))
        layers.extend([
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)

# Mobile Net Block

In [3]:
class MobileNetV2(nn.Module):
    def __init__(self, input_channel, num_classes, width_mult=1.0, inverted_residual_setting=None):
        
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        
        if inverted_residual_setting is None:
            inverted_residual_setting = [
                # t, c, n, s
                [1, 64, 2, 2],
                [6, 128, 4, 2],
                [6, 256, 6, 1],
            ]

        self.last_channel = 1280
        features = []
        
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = c
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
                
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))
        
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
#         print(x.shape)
        x = x.mean([2, 3])
        x = self.classifier(x)
        return x


In [4]:
model = MobileNetV2(3, 2300)
model

MobileNetV2(
  (features): Sequential(
    (0): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
          (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [5]:
# save only model parameters
PATH = "./basic_cnn_trial2_100.pt"
model = MobileNetV2(3, 2300)
model.load_state_dict(torch.load(PATH))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [10]:
# I have taken this implementation from recitation-6 and have made changes according to this assignment's requirement

def train(model, data_loader, test_loader, task='Classification'):
    model.train()
    
    for epoch in range(numEpochs):
        start = time.time()
        avg_loss = 0.0
        
        accuracy = 0
        total = 0
        
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)
            
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()          
            avg_loss += loss.item()
       
            if batch_num % 500 == 499:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/500))
                avg_loss = 0.0
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
            
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
        else:
            test_verify(model, test_loader)
            
        print("time: ", time.time()-start)
        PATH = "./new_" + str(epoch + 100) + ".pt"
        torch.save(model.state_dict(), PATH)

def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
#         print("...", outputs.shape)
        pred_labels = torch.max(F.softmax(outputs, dim=1), 1)[1]
        pred_labels = pred_labels.view(-1)
#         print(pred_labels)
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), 100*accuracy/total


In [7]:
train_dataset = torchvision.datasets.ImageFolder(root='./train_data/medium/', 
                                                 transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, 
                                               shuffle=True, num_workers=8)

dev_dataset = torchvision.datasets.ImageFolder(root='./validation_classification/medium/', 
                                               transform=torchvision.transforms.ToTensor())
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=100, 
                                             shuffle=True, num_workers=8)

In [12]:
numEpochs = 30
learningRate = 1e-4
weightDecay = 5e-5
 
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr =0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)

In [14]:
model.train()
model.to(device)
train(model, train_dataloader, dev_dataloader)

Epoch: 1	Batch: 500	Avg-Loss: 0.2731
Epoch: 1	Batch: 1000	Avg-Loss: 0.2649
Epoch: 1	Batch: 1500	Avg-Loss: 0.2643
Epoch: 1	Batch: 2000	Avg-Loss: 0.2755
Epoch: 1	Batch: 2500	Avg-Loss: 0.2673
Epoch: 1	Batch: 3000	Avg-Loss: 0.2638
Epoch: 1	Batch: 3500	Avg-Loss: 0.2646
Epoch: 1	Batch: 4000	Avg-Loss: 0.2641
Epoch: 1	Batch: 4500	Avg-Loss: 0.2636
Epoch: 1	Batch: 5000	Avg-Loss: 0.2629
Epoch: 1	Batch: 5500	Avg-Loss: 0.2677
Epoch: 1	Batch: 6000	Avg-Loss: 0.2670
Epoch: 1	Batch: 6500	Avg-Loss: 0.2636
Epoch: 1	Batch: 7000	Avg-Loss: 0.2578
Epoch: 1	Batch: 7500	Avg-Loss: 0.2668
Epoch: 1	Batch: 8000	Avg-Loss: 0.2607
Train Loss: 0.1680	Train Accuracy: 96.9563	Val Loss: 1.0528	Val Accuracy: 78.0048
time:  1728.1768281459808
Epoch: 2	Batch: 500	Avg-Loss: 0.2554
Epoch: 2	Batch: 1000	Avg-Loss: 0.2576
Epoch: 2	Batch: 1500	Avg-Loss: 0.2614
Epoch: 2	Batch: 2000	Avg-Loss: 0.2614
Epoch: 2	Batch: 2500	Avg-Loss: 0.2621
Epoch: 2	Batch: 3000	Avg-Loss: 0.2568
Epoch: 2	Batch: 3500	Avg-Loss: 0.2543
Epoch: 2	Batch: 4000

KeyboardInterrupt: 

In [ ]:
model.train()
model.to(device)
train(model, train_dataloader, dev_dataloader)

In [ ]:
PATH = "./basic_cnn2.pt"
torch.save(model.state_dict(), PATH)

# Test

In [17]:
mapkv = imageFolder_dataset.class_to_idx

In [18]:
mapping = {v:k for k,v in mapkv.items()}

In [19]:
def test_classify_closs(model, test_loader):
    label = []
    model.eval()
    k =0 
    for batch_num, (feats, labels) in enumerate(test_loader):
#         print(k)
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
        pred_labels = torch.max(F.softmax(outputs, dim=1), 1)[1]
        pred_labels = pred_labels.view(-1)
        k += 1
        y_cap = pred_labels[0].item()
        
        label.append(mapping[y_cap])
        
    return label

In [20]:
test_dataset = torchvision.datasets.ImageFolder(root='./test_classification/', 
                                                 transform=torchvision.transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, num_workers=8)

In [21]:
# generating output

model.to(device)
output = test_classify_closs(model, test_dataloader)

In [22]:
# to download csv file

from IPython.display import HTML
import pandas as pd
import numpy as np
df = pd.DataFrame(np.array(output), columns=['label'])
df.to_csv('submission.csv')
def create_download_link(title = "Download CSV file", filename = "data.csv"):
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')